In [91]:
import pandas as pd
import numpy as np
import datetime
import os
import re
import abc_analysis

In [92]:
services = pd.read_excel('services.xlsx', dtype={'Код услуги': str})

In [93]:
services

,Код услуги,Наименование услуги,Категория крупно,Год,Месяц,Выработка,Кол-во услуг
0,0009.000100,Подбор компрессионного трикотажа ДУОМЕД 2 клас...,расходник,2020,июн.,4100.0,1
1,0009.000100,Подбор компрессионного трикотажа ДУОМЕД 2 клас...,расходник,2020,окт.,8200.0,2
2,0009.000100,Подбор компрессионного трикотажа ДУОМЕД 2 клас...,расходник,2020,нояб.,4100.0,1
3,0010.000010,Консультация врача - терапевта первичная в амб...,консультации первичные,2020,янв.,3242010.0,2204
4,0010.000010,Консультация врача - терапевта первичная в амб...,консультации первичные,2020,февр.,3404471.0,2323
...,...,...,...,...,...,...,...
21966,100193.000150,УЗИ предстательной железы (ректальным датчиком...,УЗИ,2021,янв.,7687.5,1
21967,100193.000160,УЗИ предстательной железы и мочевого пузыря с ...,УЗИ,2020,авг.,4545.0,1
21968,100193.000160,УЗИ предстательной железы и мочевого пузыря с ...,УЗИ,2020,окт.,5050.0,1
21969,100193.000160,УЗИ предстательной железы и мочевого пузыря с ...,УЗИ,2020,дек.,9595.0,2


In [94]:
services_new = services.pivot_table(index='Код услуги', columns=['Год', 'Месяц'], values='Выработка', aggfunc=sum)

In [95]:
services_new.columns = list(map(lambda x: ' '.join([str(i) for i in x]), services_new.columns))

In [96]:
services_new.dropna(inplace=True)

In [97]:
services_new.drop('2021 февр.', axis=1, inplace=True)

In [98]:
services_new['Код'] = services_new.index

In [99]:
services_new.index = range(services_new.shape[0])

In [100]:
for column in services_new.columns:
    if column != 'Код':
        temp_abc_list = services_new[column][abc_analysis.abc_analysis(services_new[column])['Aind']].to_list()
        services_new[column + 'ABC'] = services_new[column].isin(temp_abc_list)
        

In [101]:
services_new_final = services_new[[column for column in services_new.columns if any(['ABC' in column, 'Код' in column])]]

In [102]:
services_new_final.index = services_new_final['Код']
services_new_final.drop('Код', axis=1, inplace=True)

C:\Users\AmanovRA\Anaconda3\lib\site-packages\pandas\core\frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [103]:
lst = (services_new_final == True).all(axis=1)[(services_new_final == True).all(axis=1)==True].index.to_list()

In [104]:
services_new['ABC_all'] = services_new['Код'].isin(lst)

In [105]:
services_new.columns

Index(['2020 авг.', '2020 апр.', '2020 дек.', '2020 июл.', '2020 июн.',
       '2020 мар.', '2020 мая', '2020 нояб.', '2020 окт.', '2020 сент.',
       '2020 февр.', '2020 янв.', '2021 янв.', 'Код', '2020 авг.ABC',
       '2020 апр.ABC', '2020 дек.ABC', '2020 июл.ABC', '2020 июн.ABC',
       '2020 мар.ABC', '2020 маяABC', '2020 нояб.ABC', '2020 окт.ABC',
       '2020 сент.ABC', '2020 февр.ABC', '2020 янв.ABC', '2021 янв.ABC',
       'ABC_all'],
      dtype='object')

In [106]:
services_new = services_new[['2020 авг.', '2020 апр.', '2020 дек.', '2020 июл.', '2020 июн.',
       '2020 мар.', '2020 мая', '2020 нояб.', '2020 окт.', '2020 сент.',
       '2020 февр.', '2020 янв.', '2021 янв.', 'Код', 'ABC_all']]

In [111]:
j = services_new[services_new['ABC_all']].sum()

In [112]:
k = services_new.sum()

In [114]:
j.drop(['Код', 'ABC_all'], inplace=True)
k.drop(['Код', 'ABC_all'], inplace=True)

In [117]:
(j / k) * 100

2020 авг.     66.1981
2020 апр.     70.7347
2020 дек.     68.1622
2020 июл.     66.7609
2020 июн.     66.1891
2020 мар.     71.3214
2020 мая      68.5014
2020 нояб.    68.0093
2020 окт.     68.4798
2020 сент.    68.6968
2020 февр.     71.023
2020 янв.     70.3694
2021 янв.     68.0625
dtype: object

In [116]:
k

2020 авг.     1.14271e+08
2020 апр.     6.96937e+07
2020 дек.     1.47418e+08
2020 июл.     1.03815e+08
2020 июн.     9.09219e+07
2020 мар.     1.58242e+08
2020 мая      7.81249e+07
2020 нояб.    1.34132e+08
2020 окт.     1.43292e+08
2020 сент.    1.33956e+08
2020 февр.    1.61331e+08
2020 янв.     1.42371e+08
2021 янв.     1.34693e+08
dtype: object

In [118]:
services_new_super = services.pivot_table(index='Код услуги', columns=['Год', 'Месяц'], values='Выработка', aggfunc=sum)
services_new_super.columns = list(map(lambda x: ' '.join([str(i) for i in x]), services_new.columns))

ValueError: Length mismatch: Expected axis has 14 elements, new values have 15 elements

In [121]:
services_new_super.columns = list(map(lambda x: ' '.join([str(i) for i in x]), services_new_super.columns))

In [123]:
services_new_super.fillna(0, inplace=True)

In [126]:
services_new_super['Код'] = services_new_super.index.isin(lst)

In [127]:
services_new_super

,2020 авг.,2020 апр.,2020 дек.,2020 июл.,2020 июн.,2020 мар.,2020 мая,2020 нояб.,2020 окт.,2020 сент.,2020 февр.,2020 янв.,2021 февр.,2021 янв.,Код
Код услуги,,,,,,,,,,,,,,,
0009.000100,0.0,0.0,0.0,0.0,4100.0,0.0,0.0,4100.0,8200.0,0.0,0.0,0.0,0.0,0.0,False
0010.000010,1760542.0,1949115.0,3196068.0,1271526.0,1079349.0,3821418.0,1199255.0,3275711.0,2674832.0,2337085.0,3404471.0,3242010.0,211956.0,2616504.0,True
0010.000050,478841.0,332090.0,541013.0,449482.0,333096.5,464178.0,358765.0,601760.5,537933.0,533137.0,539726.0,518672.0,51040.0,627462.0,True
0010.000070,1012458.0,1445816.0,1539720.0,532685.0,627927.0,2886162.0,709277.0,1702362.0,1618585.0,1644431.0,2233478.0,1721499.0,89895.0,1218841.0,True
0010.000110,63650.0,60705.0,114532.0,42465.0,35392.0,97945.0,30115.0,78850.0,79800.0,53081.0,84417.0,43415.0,6460.0,60800.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100193.000110,0.0,0.0,7005.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3510.0,False
100193.000120,0.0,0.0,10065.0,0.0,0.0,0.0,0.0,8625.0,24667.5,4312.0,0.0,0.0,2925.0,0.0,False
100193.000130,0.0,0.0,12187.5,0.0,0.0,0.0,0.0,23595.0,16087.5,12675.0,0.0,0.0,4100.0,21525.0,False


In [128]:
j = services_new_super[services_new_super['Код']].sum()
k = services_new.sum()
j.drop(['Код'], inplace=True)
k.drop(['Код'], inplace=True)
(j / k) * 100

2020 авг.     66.1981
2020 апр.     70.7347
2020 дек.     68.1622
2020 июл.     66.7609
2020 июн.     66.1891
2020 мар.     71.3214
2020 мая      68.5014
2020 нояб.    68.0093
2020 окт.     68.4798
2020 сент.    68.6968
2020 февр.     71.023
2020 янв.     70.3694
2021 февр.        NaN
2021 янв.     68.0625
ABC_all           NaN
dtype: object

In [131]:
((j / k) * 100).dropna().mean()

68.65451136194451

In [133]:
pd.Series(lst, name='Коды').to_excel('Коды.xlsx')

In [138]:
import random

lst = list(range(1, 101))

random.choices(lst, k=10)

[34, 59, 93, 80, 10, 86, 26, 39, 23, 74]

In [141]:
import numpy as np
import abc_analysis
arr = np.random.randint(1, 100, size=10)

abc_analysis.abc_analysis(arr)

{'ABLimit': 71,
 'BCLimit': 42,
 'ABexchanged': False,
 'Aind': [2, 4, 6],
 'Bind': [0, 1, 3],
 'Cind': [5, 7, 8, 9],
 'smallestAData': 0.6971394004308717,
 'smallestBData': 0.8636363636363639,
 'AlimitIndInInterpolation': 44,
 'BlimitIndInInterpolation': 60,
 'p': 0      0.00
 1      0.01
 2      0.02
 3      0.03
 4      0.04
        ... 
 96     0.96
 97     0.97
 98     0.98
 99     0.99
 100    1.00
 Name: Effort, Length: 101, dtype: float64,
 'ABC': 0      2.151660e-18
 1      2.161641e-02
 2      4.249183e-02
 3      6.266959e-02
 4      8.219297e-02
            ...     
 96     9.896782e-01
 97     9.918754e-01
 98     9.943041e-01
 99     9.970003e-01
 100    1.000000e+00
 Name: Yield, Length: 101, dtype: float64,
 'A': {'Effort': 0.44, 'Yield': 0.6971394004308717},
 'B': {'Effort': 0.52, 'Yield': 0.7989478085176304},
 'C': {'Effort': 0.6, 'Yield': 0.8636363636363639},
 'CleanedData': 0    42
 1    71
 2    74
 3    67
 4    91
 5    23
 6    73
 7    11
 8    22
 9    10
 dty